In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from utils.data_preprocess import * # data_split, DataLabeling, train_val_test_split
from models import *
from utils.Backtest import Backtest

In [2]:
WINDOW_SIZE = 24

In [15]:
for price in os.listdir('./data'):
    df = pd.read_csv(f'./data/{price}')
    raw_data = DataLabeling(df, WINDOW_SIZE)
    train_df, val_df, test_df = train_val_test_split(raw_data.labelled_data, rolling=100000)
    Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)
    model = CDT_1D_model(WINDOW_SIZE, len(train_df.columns)-1)
    test_model = model.model
    test_model.fit(Data.train,
               epochs=20000,
               validation_data=Data.val,
               steps_per_epoch=len(Data.train),
               validation_steps=int(0.15*len(Data.val)),
               callbacks=[create_model_checkpoint(model_name=test_model.name, save_path=f'{price[:6]}_without_TIs'),
                          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

Epoch 1/20000
11867/11867 [==============================] - 144s 12ms/step - loss: 0.7440 - accuracy: 0.5756 - val_loss: 317.7543 - val_accuracy: 0.1250
Epoch 2/20000
11867/11867 [==============================] - 142s 12ms/step - loss: 0.7331 - accuracy: 0.5899 - val_loss: 246.6564 - val_accuracy: 0.1250
Epoch 3/20000
11867/11867 [==============================] - 133s 11ms/step - loss: 0.7201 - accuracy: 0.5977 - val_loss: 95.2251 - val_accuracy: 0.1250
Epoch 4/20000
11867/11867 [==============================] - 132s 11ms/step - loss: 0.7088 - accuracy: 0.6025 - val_loss: 202.7976 - val_accuracy: 0.0000e+00
Epoch 5/20000
11867/11867 [==============================] - 146s 12ms/step - loss: 0.7045 - accuracy: 0.6036 - val_loss: 10.4446 - val_accuracy: 0.8750
Epoch 6/20000
11867/11867 [==============================] - 152s 13ms/step - loss: 0.7027 - accuracy: 0.6044 - val_loss: 20.3838 - val_accuracy: 0.0000e+00
Epoch 7/20000
11867/11867 [==============================] - 152s 13ms/

NameError: name 'exit' is not defined

In [16]:
df = pd.read_csv(f'./data/RVNBTC-5m-data.csv')
raw_data = DataLabeling(df, WINDOW_SIZE)
train_df, val_df, test_df = train_val_test_split(raw_data.labelled_data, rolling=80000)
Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)
model = CDT_1D_model(WINDOW_SIZE, len(train_df.columns)-1)
test_model = model.model
test_model.fit(Data.train,
            epochs=20000,
            validation_data=Data.val,
            steps_per_epoch=len(Data.train),
            validation_steps=int(0.15*len(Data.val)),
            callbacks=[create_model_checkpoint(model_name=test_model.name, save_path='RVNBTC_without_TIs'),
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

Epoch 1/20000
11867/11867 [==============================] - 180s 15ms/step - loss: 1.0623 - accuracy: 0.4832 - val_loss: 26.4359 - val_accuracy: 0.5833
Epoch 2/20000
11867/11867 [==============================] - 167s 14ms/step - loss: 1.0454 - accuracy: 0.5043 - val_loss: 167.6603 - val_accuracy: 0.1667
Epoch 3/20000
11867/11867 [==============================] - 153s 13ms/step - loss: 1.0295 - accuracy: 0.5114 - val_loss: 64.8361 - val_accuracy: 0.3750
Epoch 4/20000
11867/11867 [==============================] - 161s 14ms/step - loss: 1.0135 - accuracy: 0.5178 - val_loss: 10.7392 - val_accuracy: 0.1667
Epoch 5/20000
11867/11867 [==============================] - 164s 14ms/step - loss: 1.0403 - accuracy: 0.4668 - val_loss: 1.0523 - val_accuracy: 0.4583
Epoch 6/20000
11867/11867 [==============================] - 160s 13ms/step - loss: 1.0355 - accuracy: 0.4705 - val_loss: 1.0682 - val_accuracy: 0.4583
Epoch 7/20000
11867/11867 [==============================] - 164s 14ms/step - loss:

In [ ]:
# asset = Backtest(train_df, initial_balance=10000).test()
# print(asset)

balance: 7.5 | crypto: 979553.9545734212
7.5


g:\Code\Python\GitHub\Is-Technical-Indicator-Really-a-Thing-for-Deep-Learning\utils\Backtest.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Processed_Trend'] = data.Trend.replace(to_replace=1, method='bfill')
g:\Code\Python\GitHub\Is-Technical-Indicator-Really-a-Thing-for-Deep-Learning\utils\Backtest.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Previous_Trend'] = data.Processed_Trend.shift()


In [ ]:
test_model = tf.keras.models.load_model('ETHUSDT_without_TIs\CDT-1D_model')
test_model.evaluate(Data.test)

7/7 [==============================] - 0s 8ms/step - loss: 3.6387e-04 - accuracy: 1.0000


[0.00036386758438311517, 1.0]

In [14]:
tf.argmax(test_model.predict(Data.test), axis=1)

<tf.Tensor: shape=(73,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1], dtype=int64)>

In [33]:
train_df.Trend.value_counts()

1    142395
0        16
2         5
Name: Trend, dtype: int64